In [1]:
import sys

sys.path.insert(0, '../Project_1_Policy_Parsing/')
from utils_io import *

In [2]:
query_args = {'select': '*',
               'where' : 'city == "San Francisco"'}
us_df = pull_df_from_socrata('nk3m-k4s8', query_args=query_args)

pulling data in 1 chunks of 154530 rows each
pulling chunk 0
took 32.5601 seconds


## All the data

In [3]:
!pwd

/Users/okeefe/Box/USF Data Science Practicum/2020-21/Okeefe/Project_2_Resilience_Building_Classifier


In [4]:
image_ds = pd.read_csv('../../Resilience_Project/model_data_set.csv', index_col='Unnamed: 0').drop('Unnamed: 0.1', axis=1)

In [5]:
query_args = {'select': '*',
               'where' : 'city == "San Francisco"'}
us_df = pull_df_from_socrata('awyk-mq2f', query_args=query_args)

pulling data in 1 chunks of 154530 rows each
pulling chunk 0
took 50.108 seconds


In [6]:
zones = ['Residential', 'Downtown', 'Industrial', 'Production', 'Commercial', 'Business']
where_arg = [f"plu_description LIKE '%{zone}%'" for zone in zones]
where_query = ' OR '.join(where_arg)
where_query

"plu_description LIKE '%Residential%' OR plu_description LIKE '%Downtown%' OR plu_description LIKE '%Industrial%' OR plu_description LIKE '%Production%' OR plu_description LIKE '%Commercial%' OR plu_description LIKE '%Business%'"

In [7]:
query_args = {'select' : '*',
              'where'  : where_query}
us_df = pull_df_from_socrata('awyk-mq2f', query_args=query_args)

pulling data in 2 chunks of 1000000 rows each
pulling chunk 0
pulling chunk 1


ReadTimeout: HTTPSConnectionPool(host='data.bayareametro.gov', port=443): Read timed out. (read timeout=100)

In [ ]:
us_df.isna().sum(axis=0)/len(us_df)

In [ ]:
sample_df.select_dtypes('float')

In [ ]:
def make_labels(df):
    res = df[((df.plu_description.str.contains('Residential')) &
                  ~((df.plu_description.str.contains('Downtown')) |
                   (df.plu_description.str.contains('Industrial')) |
                   (df.plu_description.str.contains('Production')) |
                   (df.plu_description.str.contains('Commercial')) |
                   (df.plu_description.str.contains('Business'))
                   ))].copy()
    com = df[(((df.plu_description.str.contains('Commercial')) |
                    (df.plu_description.str.contains('Business'))) &
                   ~((df.plu_description.str.contains('Downtown')) |
                   (df.plu_description.str.contains('Industrial')) |
                   (df.plu_description.str.contains('Production')) |
                   (df.plu_description.str.contains('Residential'))
                    ))].copy()

    ind = df[(((df.plu_description.str.contains('Industrial')) | 
                  (df.plu_description.str.contains('Production'))) &
                   ~((df.plu_description.str.contains('Downtown')) |
                   (df.plu_description.str.contains('Commercial')) |
                   (df.plu_description.str.contains('Business')) |
                   (df.plu_description.str.contains('Residential'))
                   ))].copy()

    dt = df[((df.plu_description.str.contains('Downtown')) & 
                 ~((df.plu_description.str.contains('Industrial')) |
                   (df.plu_description.str.contains('Production')) |
                   (df.plu_description.str.contains('Commercial')) |
                   (df.plu_description.str.contains('Business')) |
                   (df.plu_description.str.contains('Residential'))
                  ))].copy()
    
    res["label"] = "residential"
    com["label"] = "commercial"
    ind["label"] = "industrial"
    dt["label"] = "downtown"
    
    return pd.concat([res, com, ind, dt])

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class Debug(BaseEstimator, TransformerMixin): # Note this class inherits from two different ancestors
    "Allow introspection of transformation in middle of a pipeline"

    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X):
#         self.shape = X.shape # Change this to the attribute you care about to return its state
        print(X) # Display to scree
        return X

# Classification run with all the values

In [ ]:
zones = ['Residential', 'Downtown', 'Industrial', 'Production', 'Commercial', 'Business']
where_arg = [f"plu_description LIKE '%{zone}%'" for zone in zones]
where_query = ' OR '.join(where_arg)
where_query
query_args = {'select' : '*',
              'where'  : where_query}
us_df = pull_df_from_socrata('awyk-mq2f', query_args=query_args)

In [ ]:
labeled_df = make_labels(us_df)

In [ ]:
labeled_df.head()

In [ ]:
plt.hist(labeled_df['label'])

In [ ]:
#features = ['county_fips', 'city', 'plu_county', 'plu_jurisdiction', 'max_far', 'max_dua', 'max_height', 'units_per_lot', 'minimum_lot_sqft', 'hs', 'ht', 'hm', 'of', 'ho', 'sc', 'il', 'iw', 'ih', 'rs', 'rb', 'mr', 'mt', 'me', 'acres']
features = ['county_fips', 
            'city', 
            'plu_county', 
            'plu_jurisdiction', 
            'max_far', 
            'max_dua', 
            'max_height', 
            'units_per_lot', 
            'minimum_lot_sqft', 
            'acres']
X = labeled_df[features].copy()
y = labeled_df['label'].copy()

In [ ]:
X['plu_jurisdiction'].unique()

# Todo: Find a way to associate all these data points with geometry

In [ ]:
sample_df = sample_df[:1000]
sample_df

In [ ]:
from shapely import wkt
from shapely.geometry import Point, Polygon, MultiPolygon

In [ ]:
row = sample_df['geometry'][293534]
row

In [ ]:
# wiggle room: 4 sig figs in lat/long represents ~10m. Shouldn't be a problem unless
# the building is a small single resident

def find_centroid(row):
    coords = row['coordinates'][0][0]
    
    x_cent = 0
    y_cent = 0
    
    for x, y in coords:
        x_cent += x
        y_cent += y
        
    x_cent /= len(coords)
    y_cent /= len(coords)
    
    return x_cent, y_cent

find_centroid(row)

In [ ]:
!pwd

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test             = train_test_split(X, y, stratify=y)

In [ ]:
y_train.value_counts() / len(y_train)

In [ ]:
# Source: https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# define oversampling strategy
over = RandomOverSampler(sampling_strategy={'residential':y_train.value_counts()['residential'], 'industrial':30000, 'downtown':10000, 'commercial':40000})
# fit and apply the transform
X_train, y_train = over.fit_resample(X_train, y_train)
# define undersampling strategy
under = RandomUnderSampler(sampling_strategy={'residential':100000, 'industrial':30000, 'downtown':10000, 'commercial':40000})
# fit and apply the transform
X_train, y_train = under.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts() / len(y_train)

In [ ]:
plt.hist(y_train);

In [ ]:
categorical_columns = (X.dtypes == object)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.impute import SimpleImputer

con_pipe = Pipeline([('scaler', StandardScaler()),
                     ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
                    ])

cat_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')),
                     ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True))])

from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer([('categorical', cat_pipe, categorical_columns),
                                   ('continuous', con_pipe, ~categorical_columns)
                                  ])

from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([('preprocessing', preprocessing),
                 ('rf', RandomForestClassifier())])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_pred, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_pred, y_test, labels=['commercial', 'downtown', 'industrial', 'residential']))